In [12]:
from IPython.display import clear_output

In [13]:
%pip install torch torchvision
%pip install numpy
%pip install scikit-learn
%pip install matplotlib
%pip install tqdm

clear_output()

## Contents

In this notebook, you will use a pre-trained model to do similar image search/retrieval

1. The idea is, we use a pre-trained image model to extract features / embeddings of a set of images
2. Then we use PCA to reduce the dimensions of these features (The searching will still work without PCA but dimensionality reduction helps with the speed of the model)
3. We will then pick a random image from the sample which will be our query image. We'll calculate the cosine similarity of the pca features of the query image with the pca features of all the images and pick the top images

you will use Caltech 101 object categories dataset because of the high quality of images.

In [1]:
import random

import numpy as np

import torch
from torch.utils.data import DataLoader, Subset

import torchvision
import torchvision.transforms as transforms
from torchvision import models

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
# For reproducable results. Can be skipped/commented.
torch.manual_seed(7)
random.seed(7)

## Downloading and Loading Images

In [3]:
!echo "Downloading 101_Object_Categories for image notebooks"

!curl -L -o 101_ObjectCategories.zip --progress-bar https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip?download=1
!unzip 101_ObjectCategories.zip
!mv caltech-101/101_ObjectCategories.tar.gz ./101_ObjectCategories.tar.gz
!tar -xzf 101_ObjectCategories.tar.gz
!rm 101_ObjectCategories.tar.gz
!ls

############################################################################################# 100.0%
Archive:  101_ObjectCategories.zip
replace __MACOSX/._caltech-101? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._caltech-101  
  inflating: caltech-101/101_ObjectCategories.tar.gz  

  inflating: __MACOSX/caltech-101/._101_ObjectCategories.tar.gz  
  inflating: caltech-101/show_annotation.m  
  inflating: __MACOSX/caltech-101/._show_annotation.m  
  inflating: caltech-101/Annotations.tar  
  inflating: __MACOSX/caltech-101/._Annotations.tar  
101_ObjectCategories  101_ObjectCategories.zip	caltech-101  data  del	__MACOSX  sample_data


In [4]:
batch_size = 64  # for batched. feature extraction
n_images = 1000  # num images to keep in the dataset.
n_pca_components = 300  # hyper param for number of top params to keep after PCA

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# image_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)  # dataset can be replaced with CIFAR10
image_dataset = torchvision.datasets.ImageFolder(root='101_ObjectCategories', transform=transform)
random_indices = random.sample(range(len(image_dataset)), n_images)
images_subset = Subset(image_dataset, random_indices)

image_loader = DataLoader(images_subset, batch_size, shuffle=False, num_workers=2)

## Downloading and Loading model

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.eval().to(device)

# Make sure to use output of an ALMOST last layer of the model and not the final output.
# We want the vector representations of the images. The goal here is NOT classification

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Extracting model features for all images

Extract features of all the images

## Extracting PCA features

Use PCA to reduce dimensionality of the features

## Let's get to image searching

For this section:

1. Pick a random image and it's corresponding PCA features
2. Show the picked image
3. calculate the cosine similarity of the chosen image's PCA features with PCA features of ALL other images
4. Find the indexes of top 5 most similar images(highest cosine similarity) and show those images. Hint: argsort
5. Visualize the images at these indexes. Are they similar?